https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt  
https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97


In [68]:
import os
import shutil
import numpy as np
import pandas as pd
from collections import Counter
from multiprocessing import Pool
from datasets import Dataset, DatasetDict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, f1_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import helper as hp

1: https://www.kaggle.com/code/taufiquesekh/stock-sentiment-analysis

In [69]:
dataset_filename = {
    '0': ('Tweet_filtered_TSLA.csv', ['date','text','stock']),
    '1': ('stock_tweets_filtered_TSLA.csv', ['date','text','stock']),
    '2': ('tweets_remaining_filtered_TSLA.csv', ['date','text','stock']),
}

In [70]:
DATASET_ENCODING = "ISO-8859-1"

In [71]:
dataset_path = os.path.join("", "data", dataset_filename["0"][0])
file_name = os.path.basename(dataset_path)
print(file_name)
df0 = pd.read_csv(dataset_path, names=dataset_filename["0"][1], skiprows=[0])
df0 = pd.DataFrame(df0)
df0

Tweet_filtered_TSLA.csv


,date,text,stock
0,2015-01-01,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,TSLA
1,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,TSLA
2,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,TSLA
3,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,TSLA
4,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,TSLA
...,...,...,...
1123257,2019-12-31,"I'm getting wasted on NYE, and thanks to @NHTS...",TSLA
1123258,2019-12-31,$TSLA Very Interesting dude:,TSLA
1123259,2019-12-31,If only 2020 brought more self-awareness to Te...,TSLA
1123260,2019-12-31,In 2020 I may start Tweeting out positive news...,TSLA


In [72]:
dataset_path = os.path.join("", "data", dataset_filename["1"][0])
file_name = os.path.basename(dataset_path)
print(file_name)
df1 = pd.read_csv(dataset_path, names=dataset_filename["1"][1], skiprows=[0])
df1 = pd.DataFrame(df1)
df1

stock_tweets_filtered_TSLA.csv


,date,text,stock
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA
...,...,...,...
37417,2021-09-30 02:52:38+00:00,Playing in the dirt and #chasingsunsets\n@tesl...,TSLA
37418,2021-09-30 02:40:26+00:00,I agree with @freshjiva that $TSLA ‘s EV busin...,TSLA
37419,2021-09-30 01:59:02+00:00,Hold. On. Tight. $TSLA,TSLA
37420,2021-09-30 01:38:26+00:00,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...,TSLA


In [73]:
dataset_path = os.path.join("", "data", dataset_filename["2"][0])
file_name = os.path.basename(dataset_path)
print(file_name)
df2 = pd.read_csv(dataset_path, names=dataset_filename["2"][1], skiprows=[0])
df2 = pd.DataFrame(df2)
df2

tweets_remaining_filtered_TSLA.csv


,date,text,stock
0,2020-04-09 23:56:51+00:00,$UMRX bouncing. EXTREMELY OVERSOLD #Coronaviru...,TSLA
1,2020-04-09 23:48:56+00:00,"Traders, did you secure the 💰 this week? $SPY ...",TSLA
2,2020-04-09 23:37:48+00:00,RT @vieiraUAE: Alex Vieira Tesla is the Next A...,TSLA
3,2020-04-09 23:36:09+00:00,Dow gains 285 points to cap epic rally $AAPL $...,TSLA
4,2020-04-09 23:35:16+00:00,Alex Vieira Tesla is the Next Apple Price Targ...,TSLA
...,...,...,...
60831,2020-07-16 00:06:52+00:00,"About an hour left before the ""Mid-week chart ...",TSLA
60832,2020-07-16 00:05:54+00:00,"Trading Summary - July 15, 2020\n\n7 Trades - ...",TSLA
60833,2020-07-16 00:04:31+00:00,"About an hour left before the ""Mid-week chart ...",TSLA
60834,2020-07-16 00:04:02+00:00,RT @TATrades: Quick poll - how much do you (on...,TSLA


In [74]:
# df0 = df0[:10]
# df1 = df1[:9]
# df2 = df2[:8]

In [75]:
test1_dataset = Dataset.from_pandas(df0)
test2_dataset = Dataset.from_pandas(df1)
test3_dataset = Dataset.from_pandas(df2)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'Tweet_filtered_TSLA': test1_dataset, 
    'stock_tweets_filtered_TSLA': test2_dataset,
    'tweets_remaining_filtered_TSLA': test3_dataset
})

In [76]:
folder = "data/to_predict"
if os.path.exists(folder):
    shutil.rmtree(folder)
dataset_dict.save_to_disk("data/to_predict")
dataset_dict

Saving the dataset (1/1 shards): 100%|██████████| 60836/60836 [00:00<00:00, 1842370.85 examples/s]


DatasetDict({
    Tweet_filtered_TSLA: Dataset({
        features: ['date', 'text', 'stock'],
        num_rows: 1123262
    })
    stock_tweets_filtered_TSLA: Dataset({
        features: ['date', 'text', 'stock'],
        num_rows: 37422
    })
    tweets_remaining_filtered_TSLA: Dataset({
        features: ['date', 'text', 'stock'],
        num_rows: 60836
    })
})